### taxi Q-learning

In [1]:
#강화 학습으로 올바른 위치에서 택시 승객을 태우고 내리는 문제

import gym

env = gym.make("Taxi-v3")

In [2]:
#환경 초기화

env.reset()

(129, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [3]:
#환경 초기화
env.reset()

# 각 칸에서의 경우의 수 : 6가지 status: south(0), north(1), east(2), west(3), pickup(4), dropoff(5)
print("Action Space : {}".format(env.action_space))

# 전체 경우의 수
print("State Space : {}".format(env.observation_space))

Action Space : Discrete(6)
State Space : Discrete(500)


In [4]:
# (taxi row, taxi column, passenger index, destination index)
# (택시의 행, 택시의 열, 승객위치, 목적지)

# R(0), G(1), Y(2), B(3)
state = env.encode(3, 1, 2, 3)
print("State:", state)

env.s = state

#env.render()
# 벽에 부딪히면 -1 패널티 부여
# 승객이 없을 때는 노란색, 승객이 있을 때는 녹색
# R,G,Y,B : 가능한 픽업 및 목적지 위치
# 파란색 : 승객 픽업 장소
# 보라색 : 현재 목적지

State: 331


In [5]:
# env.P 초기 보상 테이블

env.P[328]

#이동할 때마다 -1, 승객승하차 -10

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
#랜덤 방식

# 초기 상태 설정
env.s = 328  
epochs = 0
penalties, reward = 0, 0
done = False

while not done:
    action = env.action_space.sample()
#                             액션 선택
    state, reward, done, _, info = env.step(action)
#   상태    보상

    # 보상이 -10이면 패널티 1 부여
    if reward == -10:
        penalties += 1
    epochs += 1  
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 5373
Penalties incurred: 1752


c:\Python3.9.13\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [7]:
#강화학습으로 구현

import numpy as np

#q 테이블 초기화
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [8]:
%%time

import random
from IPython.display import clear_output

# 하이퍼 파라미터
alpha = 0.1 # 학습률
gamma = 0.6 # 할인율
epsilon = 0.1

# 그래프 출력을 위한 리스트
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()[0]
    epochs, penalties, reward = 0, 0, 0
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            #랜덤값이 0.1 미만이면 새로운 경로 탐색
            action = env.action_space.sample()
        else: #그렇지 않으면 기존에 학습된 경로 선택
            #학습된 q 테이블의 값 중 최대값
            action = np.argmax(q_table[state])
        next_state, reward, done, info = env.step(action)[:4]
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: total: 44.4 s
Wall time: 51.7 s


In [9]:
q_table[328]

array([ -2.39061349,  -2.27325184,  -2.41319672,  -2.36075526,
       -10.2650946 , -11.04002001])

In [10]:
from IPython.display import Video

env = gym.make("Taxi-v3", render_mode='rgb_array')

from gymnasium.experimental.wrappers import RecordVideoV0
from IPython.display import Video

# Q-learning 종료 후의 성능 측정
total_epochs, total_penalties = 0, 0
episodes = 10

env = RecordVideoV0(env, "video", name_prefix="taxi-qlearning", disable_logger=True,episode_trigger=lambda x: x % 2 == 0 )
env.reset(seed=42)

for _ in range(episodes):
    state = env.reset()[0]
    epochs, penalties, reward = 0, 0, 0
    done = False
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, _, info = env.step(action)
        if reward == -10:
            penalties += 1
        epochs += 1
    total_penalties += penalties
    total_epochs += epochs
env.close()

print(f"Results after {episodes} episodes:")
print(f"에피소드당 평균 시간 간격: {total_epochs / episodes}")
print(f"에피소드당 평균 패널티: {total_penalties / episodes}")

c:\Python3.9.13\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Results after 10 episodes:
에피소드당 평균 시간 간격: 12.9
에피소드당 평균 패널티: 0.0


In [11]:
Video("video/taxi-qlearning-episode-10.mp4", embed=True)